# Text Analysis


In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------
import string

# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [4]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

In [5]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [6]:
df_train.head(8)
df_train.shape
df_test.head()

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני..."
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...


In [7]:
df_test.head(3)

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


In [8]:
# removing all punctuations found in the list
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    text = text.replace("\n", '')

    return text

In [9]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [10]:
# Removing none hebrew chars smd mumbers
def clear_punctuations_num(df, col):
    df[col]=df[col].apply(remove_punctuations)
    df[col] = df[col].str.replace('\d+', ' ')
    df[col] = df[col].str.replace('[a-zA-Z0-9]', ' ')
    df[col] = df[col].str.replace('  ', ' ')
    
    return df

In [11]:
def decouple_data(df, label):
    y = df[label].copy()
    X = df.drop(columns=[label]).copy()
    return X, y

In [12]:
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.10, random_state = 42, shuffle=False)
    return X_train, X_test, y_train, y_test

In [13]:
df_train = clear_punctuations_num(df_train, 'story')

In [14]:
df_train.drop_duplicates(subset='story',keep = 'first')

,story,gender
0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
...,...,...
747,בקיץ האחרון יצאתי לטיול עם חברים לטיול באיזור ...,m
748,אז לפני שנה בדיוק טסתי לאמסטרדם עם שני חברים ט...,m
750,לפני חודש עברנו לדירה בבית שמש בעקבות משפחתי ה...,m
751,החוויה אותה ארצה לשתף התרחשה לפני כמה חודשים ז...,f


In [15]:
# changing the values to numerical
df_train['gender'] = df_train['gender'].replace('f', 1)
df_train['gender'] = df_train['gender'].replace('m', 0)
df_train.head()

,story,gender
0,כשחבר הזמין אותי לחול לא באמת חשבתי שזה יקרה פ...,0
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,0
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,1
3,כשהייתי ילד מטוסים היה הדבר שהכי ריתק אותי בתו...,0
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,1


In [16]:
X, y = decouple_data(df_train, 'gender')
print("X shape is:", X.shape)
print("y shape is:", y.shape)

X shape is: (753, 1)
y shape is: (753,)


In [17]:
X_train, X_test, y_train, y_test = split_data(X, y)
print("X_train shape is:", X_train.shape)
print("y_train shape is:", y_train.shape)
print("X_test shape is:", X_test.shape)
print("y_test shape is:", y_test.shape)

X_train shape is: (677, 1)
y_train shape is: (677,)
X_test shape is: (76, 1)
y_test shape is: (76,)


In [18]:
def predict(text_clf, y):
    return text_clf.predict(y)

In [19]:
# This was the best clf that didn't do any overfit, chose to look only at words with more than
# 3 characters
perceptron_clf = Pipeline([ 
        ('vect', TfidfVectorizer(ngram_range=(1,4),token_pattern  = r'(\w{3,20})\b',max_df = 0.50)),
        ('clf', Perceptron(penalty='l1',
                              alpha=1e-7, random_state=42, tol=None)),
    ])
    
perceptron_clf.fit(X_train.story, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.5, ngram_range=(1, 4),
                                 token_pattern='(\\w{3,20})\\b')),
                ('clf',
                 Perceptron(alpha=1e-07, penalty='l1', random_state=42,
                            tol=None))])

In [20]:
y_pred=predict(perceptron_clf, X_test.story)
confusion_matrix(y_test,y_pred)
print("The model's macro f1 score is:",f1_score(y_test, y_pred, average='macro'))
print('\n')
print(metrics.classification_report(y_test, y_pred))

array([[55,  3],
       [ 9,  9]], dtype=int64)

The model's macro f1 score is: 0.7508196721311475


              precision    recall  f1-score   support

           0       0.86      0.95      0.90        58
           1       0.75      0.50      0.60        18

    accuracy                           0.84        76
   macro avg       0.80      0.72      0.75        76
weighted avg       0.83      0.84      0.83        76



In [21]:
# cleaning the data before predicting as done on the training data
df_test = clear_punctuations_num(df_test, 'story')
y_pred_test=predict(perceptron_clf, df_test.story)
y_pred_test = pd.DataFrame(y_pred_test, columns = ['y_pred_test'])
y_pred_test['y_pred_test'].value_counts()
# maping the values back to male and female
df_test['predicted_category'] = y_pred_test['y_pred_test'].replace(1, 'f')
df_test['predicted_category'] = y_pred_test['y_pred_test'].replace(0, 'm')
df_test.drop(columns='story',inplace=True)


0    242
1     81
Name: y_pred_test, dtype: int64

In [22]:
df_predicted = df_test.copy()
df_predicted

,test_example_id,predicted_category
0,0,m
1,1,m
2,2,m
3,3,m
4,4,1
...,...,...
318,318,m
319,319,m
320,320,m
321,321,m
